<a href="https://colab.research.google.com/github/onewon1234/AI_DL_Project/blob/KLUE-RoBERTa/KLUE-RoBERTa/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import itertools
from itertools import permutations
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)

In [4]:
print(torch.cuda.is_available())

True


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd /content/drive/MyDrive/강의/AI를 위한 딥러닝/AI_DL_Project/code

/content/drive/MyDrive/강의/AI를 위한 딥러닝/AI_DL_Project/code


In [7]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [8]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7351 entries, 0 to 7350
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          7351 non-null   object
 1   sentence_0  7351 non-null   object
 2   sentence_1  7351 non-null   object
 3   sentence_2  7351 non-null   object
 4   sentence_3  7351 non-null   object
 5   answer_0    7351 non-null   int64 
 6   answer_1    7351 non-null   int64 
 7   answer_2    7351 non-null   int64 
 8   answer_3    7351 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 517.0+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3,answer_0,answer_1,answer_2,answer_3
0,TRAIN_0000,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.,"이러한 특성은 유권자들에게 신뢰를 제공하며, 민주적 참여를 촉진하는 데 기여할 수 있다.",결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다.,0,3,1,2
1,TRAIN_0001,줄거리 자동 생성의 인공지능 알고리즘은 대량의 텍스트 데이터를 분석하여 핵심 정보를...,"결과적으로, 이러한 기술은 사용자에게 신속하고 효율적인 정보 전달을 가능하게 한다.",생성된 줄거리는 원본 텍스트의 의미를 유지하면서도 간결하게 요약된 형태로 제공된다.,"이 알고리즘은 자연어 처리 기술을 활용하여 문맥을 이해하고, 주요 사건과 등장인물을...",0,3,2,1
2,TRAIN_0002,"마지막으로, 키친타올을 보관할 때는 쉽게 접근할 수 있는 곳에 두어 낭비를 방지하는...",재사용 가능한 천이나 스펀지를 활용하면 키친타올의 필요성을 줄일 수 있다.,물기를 제거할 때는 가볍게 눌러주어 과도한 사용을 피할 수 있다.,키친타올을 절약하는 첫걸음은 필요한 양만큼만 사용하는 것이다.,3,2,1,0
3,TRAIN_0003,책의 페이지가 손상되지 않도록 수직으로 세워 두거나 평평하게 눕혀 보관하는 것이 좋다.,"정기적으로 먼지를 털어내고, 곰팡이나 해충의 발생 여부를 점검하는 것이 중요하다.",종이책은 직사광선이 닿지 않는 서늘하고 건조한 장소에 보관해야 한다.,"필요할 경우, 책을 보호하기 위해 커버를 씌우거나 전용 보관함에 넣는 방법도 고려할...",2,0,1,3
4,TRAIN_0004,"인공지능 모델은 반복적인 실험을 통해 지속적으로 학습하며, 이를 통해 발견의 정확성...",인공지능은 대량의 데이터를 분석하여 숨겨진 패턴과 상관관계를 발견하는 데 강력한 도...,"결국, 인공지능의 지원은 과학적 발견의 속도와 효율성을 혁신적으로 변화시킬 수 있는...",이러한 분석 결과는 연구자들에게 새로운 가설을 제시하고 실험 설계를 개선하는 데 기...,1,3,0,2


In [9]:
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          1780 non-null   object
 1   sentence_0  1780 non-null   object
 2   sentence_1  1780 non-null   object
 3   sentence_2  1780 non-null   object
 4   sentence_3  1780 non-null   object
dtypes: object(5)
memory usage: 69.7+ KB


,ID,sentence_0,sentence_1,sentence_2,sentence_3
0,TEST_0000,"자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만, 이 둘의 공존 가능성도 탐...","결정론은 모든 사건이 원인과 결과의 연쇄에 의해 발생한다고 주장하며, 이는 인간의 ...",그러나 인간의 인식과 선택 과정에서 나타나는 복잡성과 예측 불가능성은 자유 의지의 ...,"결국, 자유 의지와 결정론은 서로를 배제하기보다는, 인간 경험의 다양한 측면을 설명..."
1,TEST_0001,사회적 낙인은 개인의 자아 존중감에 부정적인 영향을 미친다.,"건강 불평등은 이러한 낙인으로 인해 더욱 심화되며, 특정 집단이 의료 서비스 접근에...","결국, 사회적 낙인과 건강 불평등은 서로 연결되어 있으며, 이를 해결하기 위한 포괄...","낙인으로 인해 사람들은 사회적 고립을 경험하고, 이는 정신적 및 신체적 건강에 악영..."
2,TEST_0002,글쓰기 능력을 키우기 위해서는 꾸준한 연습이 필수적이다.,"마지막으로, 독서를 통해 다른 작가들의 기법을 배우는 것은 창의력을 자극하는 데 도...",피드백을 받는 과정은 글의 질을 향상시키는 중요한 요소로 작용한다.,다양한 주제에 대해 글을 써보면 자신의 스타일과 강점을 발견할 수 있다.
3,TEST_0003,작은 공간에서도 효율적으로 사용할 수 있어 집안의 혼잡함을 줄여준다.,정기적으로 내용을 점검하면 필요 없는 물건을 정리할 수 있다.,각 칸을 활용하여 카테고리별로 물건을 나누면 찾기 쉬워진다.,다용도 수납함은 다양한 물건을 정리하는 데 유용하다.
4,TEST_0004,음악은 특정 문화의 가치와 전통을 반영하는 중요한 매체이다.,이러한 음악적 표현은 공동체의 소속감을 증진시키는 역할을 한다.,각 문화는 고유한 음악적 요소를 통해 정체성을 형성하고 강화한다.,"결국, 음악은 개인과 집단의 문화적 정체성을 이해하는 데 필수적인 요소로 작용한다."


In [10]:
# 텍스트 정제
def clean_text(text):
  # 특수문자 제거
  text = re.sub(r'[^\w\s]', '', text)
  # 소문자 변환: 한글에는 무의미
  text = text.lower()
  # 불필요한 공백 제거
  text = ' '.join(text.split())
  return text

In [11]:
# 정제 전
print(train['sentence_0'][0])
print(test['sentence_0'][0])

블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다.
자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만, 이 둘의 공존 가능성도 탐구할 가치가 있다.


In [12]:
# 텍스트 정제
for i in range(4):
    train[f'sentence_{i}'] = train[f'sentence_{i}'].apply(clean_text)
    test[f'sentence_{i}'] = test[f'sentence_{i}'].apply(clean_text)

In [13]:
# 정제 후
print(train['sentence_0'][0])
print(test['sentence_0'][0])

블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다
자유 의지와 결정론은 서로 상충하는 개념으로 여겨지지만 이 둘의 공존 가능성도 탐구할 가치가 있다


In [14]:
## ✅ Pairwise 전처리 함수
def preprocess_pairwise(df):
    data = []
    for _, row in df.iterrows():
        sentences = [row[f'sentence_{i}'] for i in range(4)]
        answer = [row[f'answer_{i}'] for i in range(4)]
        ordered = [sentences[i] for i in answer]
        positive_pairs = [(ordered[i], ordered[i+1]) for i in range(3)]
        all_pairs = list(itertools.permutations(sentences, 2))
        for s1, s2 in all_pairs:
            label = 1 if (s1, s2) in positive_pairs else 0
            data.append({'sentence1': s1, 'sentence2': s2, 'label': label})
    return pd.DataFrame(data)

In [15]:
# 전처리 모듈 사용
pairwise_df = preprocess_pairwise(train)

In [16]:
pairwise_df.info()
pairwise_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88212 entries, 0 to 88211
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence1  88212 non-null  object
 1   sentence2  88212 non-null  object
 2   label      88212 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


,sentence1,sentence2,label
0,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,이러한 특성은 유권자들에게 신뢰를 제공하며 민주적 참여를 촉진하는 데 기여할 수 있다,0
1,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,0
2,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,각 투표는 변경 불가능한 기록으로 저장되어 조작의 가능성을 원천적으로 차단한다,1
3,이러한 특성은 유권자들에게 신뢰를 제공하며 민주적 참여를 촉진하는 데 기여할 수 있다,블록체인 기술은 투표 과정의 투명성을 크게 향상시킬 수 있다,0
4,이러한 특성은 유권자들에게 신뢰를 제공하며 민주적 참여를 촉진하는 데 기여할 수 있다,결과적으로 블록체인 기반의 투표 시스템은 공정하고 신뢰할 수 있는 선거 환경을 조성...,1


In [17]:
# 클래스 균형 확인
pairwise_df['label'].value_counts()

,count
label,
0,66159
1,22053


In [18]:
# 클래스별 샘플 수
num_zeros = pairwise_df['label'].value_counts()[0]
num_ones = pairwise_df['label'].value_counts()[1]

# 클래스 비율 기반 가중치 (클래스 1이 적으므로 더 큰 가중치)
total = num_zeros + num_ones
weight_for_0 = total / (2 * num_zeros)
weight_for_1 = total / (2 * num_ones)

class_weights = torch.tensor(
    [weight_for_0, weight_for_1],
    dtype=torch.float
)
print("Class Weights:", class_weights)

Class Weights: tensor([0.6667, 2.0000])


# 2. 데이터셋 구성

In [19]:
MAX_TOKEN_LENGTH = 128

In [20]:
# 데이터셋 클래스 정의
class SentencePairDataset(Dataset):
    """
    문장 쌍을 받아 BERT 입력 형식으로 변환하는 PyTorch Dataset 클래스

    Args:
        texts (List[Tuple[str, str]]): (문장1, 문장2) 형태의 튜플 리스트
        labels (List[int]): 문장 순서가 맞는지 여부를 나타내는 정수 레이블 (예: 0 또는 1)
        tokenizer (transformers.PreTrainedTokenizer): HuggingFace 토크나이저 객체
        max_length (int): 토큰 최대 길이 (default=128)
    """
    def __init__(self, dataframe, tokenizer, max_length=MAX_TOKEN_LENGTH):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        encoding = self.tokenizer(
            row['sentence1'],
            row['sentence2'],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': torch.tensor(row['label'], dtype=torch.long)
        }

        if 'token_type_ids' in encoding:
            item['token_type_ids'] = encoding['token_type_ids'][0]

        return item

# 3. 모델 아키텍쳐

In [21]:
class SentencePairModel(nn.Module):
    def __init__(self, model_name, num_labels=2, class_weights=None):
        super(SentencePairModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        if class_weights is not None:
            self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)
        else:
            self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            output_hidden_states=True,
                            return_dict=True)

        # 마지막 4개 hidden layer 평균
        hidden_states = outputs.hidden_states  # Tuple of (layer_num, batch, seq_len, hidden)
        last_four = torch.stack(hidden_states[-4:])     # shape: (4, batch, seq_len, hidden)
        avg_hidden = torch.mean(last_four, dim=0)       # shape: (batch, seq_len, hidden)
        cls_output = avg_hidden[:, 0]                   # [CLS] 위치만 추출

        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {'loss': loss, 'logits': logits}

# 4. 학습 코드

In [22]:
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [23]:
# 데이터 분할: 학습/검증 (예: 8:2)
train_df, val_df = train_test_split(pairwise_df, test_size=0.2, stratify=pairwise_df['label'], random_state=42)

# dataset 생성
train_dataset = SentencePairDataset(train_df, tokenizer, max_length=MAX_TOKEN_LENGTH)
val_dataset = SentencePairDataset(val_df, tokenizer, max_length=MAX_TOKEN_LENGTH)

In [24]:
train_dataset[0]

{'input_ids': tensor([    0,  5268,  2470,  6881,  4392,  2259, 15259,  2079,  1754,  2522,
          5035,  8309,  2170,  3653, 12462,     2,  6261,  2052,  5098,  2119,
         11700,  2200, 14368,  6620,  2088,  1123,  2052,  5373,  2170,  5984,
          2205,  2318,   822,  2227,  2275,  3605,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [25]:
val_dataset[0]

{'input_ids': tensor([    0,  3731, 11187,  3738,  2079,  5211,  3962,  2259,  3839,  2470,
          5629,  2470,  5230,  2333,  2048,  2079, 11920,  2200,  8045,     2,
         18772,  2266, 14486,  2073,  5211,  2079,  3962,  2522,  6747,  2138,
          3923,  2205,  2259,   842,  5588, 31221,  4038,  3993, 28674,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [26]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 가중치를 모델과 같은 device로 이동
class_weights = class_weights.to(device)

# 모델 인스턴스 생성
model = SentencePairModel("klue/roberta-base", class_weights=class_weights).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=1)

    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds)
    }

In [29]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',               # 체크포인트와 결과 저장 폴더
    num_train_epochs=3,                   # 학습 epoch 수
    per_device_train_batch_size=16,       # 학습 시 디바이스당 배치 사이즈
    per_device_eval_batch_size=64,        # 검증 시 디바이스당 배치 사이즈
    gradient_accumulation_steps=2,
    warmup_steps=500,                     # 워밍업 단계에서 학습률 선형 증가
    weight_decay=0.01,                    # AdamW optimizer의 weight decay 값
    logging_dir='./logs',                 # 로깅 파일 저장 경로
    logging_steps=10,                     # 로그를 몇 스텝마다 남길지
    save_strategy='epoch',                # epoch마다 체크포인트 저장
    eval_strategy='epoch',
    save_total_limit=1,                    # 체크포인트 최대 저장 개수 (가장 최근 것만 유지)

    # EarlyStoppingCallback 필수 옵션
    load_best_model_at_end=True,           # 학습이 끝난 뒤 최적 모델 불러오기
    metric_for_best_model='accuracy',      # 대회 평가 산식 monitor
    greater_is_better=True                 # 메트릭이 클수록 좋은 경우 True
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
   )

# 학습 실행
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: violay2002 (violay2002-chung-ang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.374300,0.380149,0.825540,0.706186,0.609893,0.838585
2,0.195400,0.345547,0.901547,0.812601,0.775216,0.853775


TrainOutput(global_step=6615, training_loss=0.32194705507742666, metrics={'train_runtime': 5171.3255, 'train_samples_per_second': 40.939, 'train_steps_per_second': 1.279, 'total_flos': 0.0, 'train_loss': 0.32194705507742666, 'epoch': 2.998866470188166})

In [30]:
# 에폭 마지막 또는 전체 학습 후 평가
eval_results = trainer.evaluate()
print("Validation Results:", eval_results)

Validation Results: {'eval_loss': 0.3455466628074646, 'eval_accuracy': 0.9015473558918551, 'eval_f1': 0.812601143596936, 'eval_precision': 0.7752161383285303, 'eval_recall': 0.8537746542734074, 'eval_runtime': 126.2677, 'eval_samples_per_second': 139.727, 'eval_steps_per_second': 2.186, 'epoch': 2.998866470188166}


In [31]:
# 모델 저장
trainer.save_model("./results/final_model")

# 5. 평가 및 추론

In [32]:
def predict_order(sent1, sent2, model, tokenizer, device='cpu'):
    model.eval()
    inputs = tokenizer(
        sent1,
        sent2,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=MAX_TOKEN_LENGTH
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs['logits'], dim=1)

    return predictions, predictions.argmax().item()